 # 天池贷款违约预测竞赛 - LightGBM Baseline



 **竞赛链接**：[天池-零基础入门金融风控-贷款违约预测](https://tianchi.aliyun.com/competition/entrance/531830/information)



 **项目背景**：

 本项目旨在构建一个机器学习模型，根据借款人的基本信息（如年收入、工作年限、信用等级等）来预测其是否会发生贷款违约（isDefault）。



 **主要流程**：

 1. **环境准备**：导入库。

 2. **数据读取**：加载并合并数据。

 3. **数据探索 (EDA)**：新增步骤，查看数据长什么样、有什么类型、缺失情况如何。

 4. **数据清洗**：根据探索结果填充缺失值。

 5. **特征工程**：将文本特征（如 '10+ years'）处理为模型能读懂的数字。

 6. **模型训练**：5折交叉验证训练 LightGBM。

 7. **结果输出**：生成提交文件。

 ## 1. 导入需要的包



 我们需要：

 * `pandas`: 数据表格处理。

 * `numpy`: 数值计算。

 * `lightgbm`: 核心算法模型。

 * `sklearn`: 机器学习工具箱（交叉验证、编码器等）。

In [1]:
import os
import gc
import re
import time
import warnings
import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

# 设置显示选项，方便在 Notebook 中查看更多列
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
# 忽略一些版本兼容性的警告
warnings.filterwarnings("ignore")


 ## 2. 导入数据与合并



 为了方便统一处理（比如统一填充缺失值、统一编码），我们先把训练集和测试集“纵向”合并在一起，处理完后再分开。

In [2]:
def load_data(train_path, test_path):
    """读取训练集和测试集"""
    print(f"正在读取数据...\n训练集: {train_path}\n测试集: {test_path}")
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    print(f"训练集大小: {train_df.shape}")
    print(f"测试集大小: {test_df.shape}")
    return train_df, test_df

TRAIN_FILE = "./train.csv"
TEST_FILE = "./testA.csv"

# 1. 读取数据
train_data, test_data = load_data(TRAIN_FILE, TEST_FILE)

# 2. 提取并分离标签（isDefault），因为它只存在于训练集
target = train_data['isDefault']
train_data = train_data.drop(['isDefault'], axis=1)

# 3. 合并数据
data = pd.concat([train_data, test_data], ignore_index=True)
print(f"合并后数据总大小: {data.shape}")


正在读取数据...
训练集: ./train.csv
测试集: ./testA.csv
训练集大小: (800000, 47)
测试集大小: (200000, 46)
合并后数据总大小: (1000000, 46)


 ## 3. 数据探索 (Exploratory Data Analysis)



 在动手处理数据之前，我们必须先“看”一眼数据。这一步非常关键，它决定了我们后续如何清洗数据。



 我们主要看三点：

 1. **数据预览 (`head`)**: 数据长什么样？有哪些列？

 2. **数据类型 (`info`)**: 哪些是数字？哪些是文本（Object）？文本通常需要转换。

 3. **缺失情况 (`isnull`)**: 哪些列缺数据？缺得多不多？

In [3]:
print("--- 1. 数据预览 (前5行) ---")
print(data.head())

print("\n--- 2. 数据信息 (列类型与内存) ---")
# 这里可以看出哪些列是 object (字符串)，比如 grade, subGrade, employmentLength
data.info()

print("\n--- 3. 数值型特征统计描述 ---")
# 这里可以看均值、最大值、最小值，帮助发现异常值
print(data.describe())


--- 1. 数据预览 (前5行) ---
   id  loanAmnt  term  interestRate  installment grade subGrade  \
0   0   35000.0     5         19.52       917.97     E       E2   
1   1   18000.0     5         18.49       461.90     D       D2   
2   2   12000.0     5         16.99       298.17     D       D3   
3   3   11000.0     3          7.26       340.96     A       A4   
4   4    3000.0     3         12.99       101.07     C       C2   

   employmentTitle employmentLength  homeOwnership  annualIncome  \
0            320.0          2 years              2      110000.0   
1         219843.0          5 years              0       46000.0   
2          31698.0          8 years              0       74000.0   
3          46854.0        10+ years              1      118000.0   
4             54.0              NaN              1       29000.0   

   verificationStatus   issueDate  purpose  postCode  regionCode    dti  \
0                   2  2014-07-01        1     137.0          32  17.05   
1               

 ### 3.1 专门查看缺失值

 我们统计每一列缺失的比例，以此决定填充策略。

In [4]:
def check_missing(df):
    """统计缺失值比例"""
    missing = df.isnull().sum()
    missing = missing[missing > 0] # 只显示有缺失的
    missing_ratio = missing / len(df)
    
    missing_df = pd.DataFrame({'缺失数量': missing, '缺失比例': missing_ratio})
    missing_df = missing_df.sort_values(by='缺失比例', ascending=False)
    
    if not missing_df.empty:
        print("\n存在缺失值的列：")
        print(missing_df)
    else:
        print("无缺失值")
    return missing_df

print("--- 缺失值分析 ---")
miss_stats = check_missing(data)


--- 缺失值分析 ---

存在缺失值的列：
                     缺失数量      缺失比例
n11                 87327  0.087327
employmentLength    58541  0.058541
n8                  50382  0.050382
n14                 50381  0.050381
n13                 50381  0.050381
n12                 50381  0.050381
n9                  50381  0.050381
n0                  50381  0.050381
n1                  50381  0.050381
n2                  50381  0.050381
n3                  50381  0.050381
n5                  50381  0.050381
n6                  50381  0.050381
n7                  50381  0.050381
n10                 41633  0.041633
n4                  41633  0.041633
revolUtil             658  0.000658
pubRecBankruptcies    521  0.000521
dti                   300  0.000300
title                   1  0.000001
postCode                1  0.000001
employmentTitle         1  0.000001


 ## 4. 数据清洗 (缺失值填充)



 根据上面的分析，我们制定如下策略：

 * **`employmentLength`**: 缺失可能代表无工作，填充为 "0"（稍后转数字）。

 * **`dti`, `revolUtil` 等连续数值**: 使用均值（mean）填充，保持数据分布大概不变。

 * **`employmentTitle` 等类别数值**: 使用众数（mode，出现最多的数）填充。

 * **匿名特征 `n0`-`n14`**: 统一用众数填充。

In [5]:
# 1. 就业年限：暂时填 0，后面特征工程会统一处理格式
data['employmentLength'] = data['employmentLength'].fillna(0)

# 2. 匿名特征 n11, n12
data['n11'] = data['n11'].fillna(0)
data['n12'] = data['n12'].fillna(0)

# 3. 众数填充 (适用于类别型或离散型特征)
for col in ['employmentTitle', 'postCode', 'title']:
    if data[col].isnull().sum() > 0:
        data[col] = data[col].fillna(data[col].mode()[0])

# 4. 均值填充 (适用于连续型数值特征)
for col in ['dti', 'pubRecBankruptcies', 'revolUtil']:
    if data[col].isnull().sum() > 0:
        data[col] = data[col].fillna(data[col].mean())

# 5. 剩余的匿名特征 (n0-n14) 统一众数填充
numerical_features = list(data.select_dtypes(exclude=['object']).columns)
no_name_list = [col for col in numerical_features if col.startswith("n") and data[col].isnull().sum() > 0]

for col in no_name_list:
    data[col] = data[col].fillna(data[col].mode()[0])

print("缺失值填充完毕。再次检查：")
check_missing(data)


缺失值填充完毕。再次检查：
无缺失值


,缺失数量,缺失比例


 ## 5. 特征工程 (关键步骤)



 模型无法直接理解字符串。我们需要将 `object` 类型的列转换为 `int` 或 `float`。



 * **`employmentLength`**: "10+ years" -> 10, "< 1 year" -> 0

 * **`earliesCreditLine`**: "Aug-2001" -> 2001 (只取年份)

 * **`grade`, `subGrade`**: "A" -> 0, "B" -> 1 (标签编码)

In [6]:
# --- 1. 处理 EmploymentLength (使用正则稳健版) ---
def employment_length_to_int(s):
    """将 '10+ years', '< 1 year' 等清洗为整数"""
    if pd.isnull(s) or str(s) == 'nan':
        return 0
    s = str(s).strip()
    if '< 1' in s: 
        return 0
    # 提取数字
    numbers = re.findall(r'\d+', s)
    if numbers:
        return int(numbers[0])
    else:
        return 0

data['employmentLength'] = data['employmentLength'].apply(employment_length_to_int)
print(f"EmploymentLength 处理后样本: {data['employmentLength'].unique()[:10]}")

# --- 2. 处理 EarliesCreditLine (提取年份) ---
# 格式通常是 "月-年"，我们取最后4位作为年份
data['earliesCreditLine'] = data['earliesCreditLine'].apply(lambda s: int(str(s)[-4:]))
print(f"EarliesCreditLine 处理后样本: {data['earliesCreditLine'].unique()[:5]}")

# --- 3. 处理 Grade 和 SubGrade (Label Encoding) ---
for col in ['grade', 'subGrade']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))
    print(f"{col} 编码完成。")

# --- 4. 删除 issueDate ---
# Baseline 暂不使用此列
if 'issueDate' in data.columns:
    data = data.drop(['issueDate'], axis=1)

# --- 5. 最终检查数据类型 ---
print("\n特征工程结束，检查是否还有非数值列：")
print(data.dtypes[data.dtypes == 'object'])


EmploymentLength 处理后样本: [ 2  5  8 10  0  7  9  1  3  4]
EarliesCreditLine 处理后样本: [2001 2002 2006 1999 1977]
grade 编码完成。
subGrade 编码完成。

特征工程结束，检查是否还有非数值列：
Series([], dtype: object)


 ## 6. 模型训练 (Model Training)



 * **模型**: LightGBM (GBDT的变种，速度快精度高)。

 * **验证**: Stratified 5-Fold CV (分层5折交叉验证)，确保验证集和训练集的正负样本比例一致。

In [7]:
# 1. 重新拆分回 训练集 和 测试集
train_features = data[:len(target)]
test_features = data[len(target):]

print(f"\n训练集维度: {train_features.shape}")
print(f"测试集维度: {test_features.shape}")

# 2. LightGBM 参数配置
param_lgb = {
    'num_leaves': 31,
    'max_depth': -1,
    'learning_rate': 0.05,
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'is_unbalance': True,        # 针对银行违约数据的不平衡问题
    'boost_from_average': False,
    'force_row_wise': True,      # 加速训练
    'verbose': -1,
    'seed': 2023
}

# 3. 交叉验证训练
nfold = 5
skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=2023)

oof_preds = np.zeros(len(train_features))
test_preds = np.zeros((len(test_features), nfold))

print(f"\n开始 {nfold} 折交叉验证训练...")
start_time = time.time()

for i, (train_index, valid_index) in enumerate(skf.split(train_features, target)):
    print(f"--- Fold {i + 1} ---")
    
    X_train, y_train = train_features.iloc[train_index], target.iloc[train_index]
    X_valid, y_valid = train_features.iloc[valid_index], target.iloc[valid_index]
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid)
    
    clf = lgb.train(
        param_lgb,
        dtrain,
        num_boost_round=5000,
        valid_sets=[dvalid],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=100)
        ]
    )
    
    oof_preds[valid_index] = clf.predict(X_valid, num_iteration=clf.best_iteration)
    test_preds[:, i] = clf.predict(test_features, num_iteration=clf.best_iteration)

end_time = time.time()
print(f"\n训练结束，耗时: {end_time - start_time:.2f} 秒")



训练集维度: (800000, 45)
测试集维度: (200000, 45)

开始 5 折交叉验证训练...
--- Fold 1 ---
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.722347
[200]	valid_0's auc: 0.727321
[300]	valid_0's auc: 0.728795
[400]	valid_0's auc: 0.729503
[500]	valid_0's auc: 0.729952
[600]	valid_0's auc: 0.730482
[700]	valid_0's auc: 0.730725
[800]	valid_0's auc: 0.730862
Early stopping, best iteration is:
[810]	valid_0's auc: 0.73087
--- Fold 2 ---
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.718795
[200]	valid_0's auc: 0.724024
[300]	valid_0's auc: 0.725661
[400]	valid_0's auc: 0.726125
[500]	valid_0's auc: 0.726559
[600]	valid_0's auc: 0.726826
[700]	valid_0's auc: 0.727152
[800]	valid_0's auc: 0.72736
[900]	valid_0's auc: 0.727525
Early stopping, best iteration is:
[931]	valid_0's auc: 0.727555
--- Fold 3 ---
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.723158
[200]	valid_0's auc: 0.728025
[300]	valid_0's 

 ## 7. 结果评估与提交



 AUC (Area Under Curve) 是金融风控最常用的指标。

 * 0.5 = 瞎猜

 * 1.0 = 完美预测

 * 通常 > 0.7 就算可用，> 0.8 就很不错了。

In [ ]:
# 计算线下验证集得分
score = roc_auc_score(target, oof_preds)
print(f"\n=== 最终 CV AUC 分数: {score:.5f} ===")

# 生成提交文件
submission = pd.DataFrame()
submission['id'] = test_data['id'] if 'id' in test_data.columns else range(len(test_preds))
submission['isDefault'] = test_preds.mean(axis=1)

output_file = 'submit_lgb.csv'
submission.to_csv(output_file, index=False)
print(f"提交文件已保存至: {output_file}")


=== 最终 CV AUC 分数: 0.72931 ===
提交文件已保存至: submit.csv
